In [2]:
from sklearn.preprocessing import LabelEncoder,StandardScaler
import pickle
import numpy as np
import pandas as pd
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import GradientBoostingRegressor

In [3]:
data=pd.read_pickle('my_data.pkl')
data

,id,loan_amnt,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,...,purpose,addr_state,dti,open_acc,total_acc,initial_list_status,last_fico_range_high,last_fico_range_low,application_type,tax_liens
0,66615254,7200.0,36 months,17.27,257.67,D,9 years,OWN,20000.0,Verified,...,debt_consolidation,MA,13.21,8.0,17.0,w,549.0,545.0,Individual,0.0
1,68366850,16000.0,36 months,5.32,481.84,A,8 years,MORTGAGE,96000.0,Not Verified,...,debt_consolidation,MN,14.45,23.0,38.0,w,744.0,740.0,Individual,0.0
2,68406994,8000.0,36 months,7.91,250.36,A,5 years,OWN,66500.0,Verified,...,home_improvement,TX,29.22,10.0,13.0,w,799.0,795.0,Individual,0.0
3,68577378,10000.0,36 months,8.49,315.63,B,4 years,OWN,35000.0,Not Verified,...,credit_card,MS,33.54,12.0,30.0,w,669.0,665.0,Individual,0.0
4,68596525,24000.0,60 months,13.99,558.32,C,10+ years,MORTGAGE,115000.0,Verified,...,debt_consolidation,IN,17.71,11.0,32.0,w,684.0,680.0,Individual,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12935,90521566,6000.0,36 months,9.49,192.17,B,10+ years,MORTGAGE,50000.0,Source Verified,...,credit_card,VA,12.48,8.0,10.0,w,684.0,680.0,Individual,0.0
12936,90681696,12000.0,60 months,14.49,282.28,C,10+ years,OWN,122000.0,Not Verified,...,credit_card,MA,7.46,5.0,21.0,f,659.0,655.0,Individual,0.0
12937,90641703,15000.0,36 months,11.49,494.57,B,1 year,OWN,70000.0,Not Verified,...,debt_consolidation,CA,27.04,19.0,30.0,w,669.0,665.0,Individual,0.0
12938,90441274,12000.0,36 months,13.99,410.08,C,1 year,RENT,47000.0,Source Verified,...,credit_card,MT,31.79,13.0,28.0,f,754.0,750.0,Individual,0.0


In [4]:
data=data.drop('id',axis=1)

In [5]:
data['application_type']=data['application_type'].replace({'Individual':0, 'Joint App':1})

C:\Users\shrey\AppData\Local\Temp\ipykernel_7256\617229969.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['application_type']=data['application_type'].replace({'Individual':0, 'Joint App':1})


In [10]:
data['initial_list_status']=data['initial_list_status'].replace({'w':0,'f':1})
data

C:\Users\shrey\AppData\Local\Temp\ipykernel_7256\580542651.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['initial_list_status']=data['initial_list_status'].replace({'w':0,'f':1})


,loan_amnt,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,dti,open_acc,total_acc,initial_list_status,last_fico_range_high,last_fico_range_low,application_type,tax_liens
0,7200.0,36 months,17.27,257.67,D,9 years,OWN,20000.0,Verified,0,debt_consolidation,MA,13.21,8.0,17.0,0,549.0,545.0,0,0.0
1,16000.0,36 months,5.32,481.84,A,8 years,MORTGAGE,96000.0,Not Verified,1,debt_consolidation,MN,14.45,23.0,38.0,0,744.0,740.0,0,0.0
2,8000.0,36 months,7.91,250.36,A,5 years,OWN,66500.0,Verified,1,home_improvement,TX,29.22,10.0,13.0,0,799.0,795.0,0,0.0
3,10000.0,36 months,8.49,315.63,B,4 years,OWN,35000.0,Not Verified,1,credit_card,MS,33.54,12.0,30.0,0,669.0,665.0,0,0.0
4,24000.0,60 months,13.99,558.32,C,10+ years,MORTGAGE,115000.0,Verified,1,debt_consolidation,IN,17.71,11.0,32.0,0,684.0,680.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12935,6000.0,36 months,9.49,192.17,B,10+ years,MORTGAGE,50000.0,Source Verified,1,credit_card,VA,12.48,8.0,10.0,0,684.0,680.0,0,0.0
12936,12000.0,60 months,14.49,282.28,C,10+ years,OWN,122000.0,Not Verified,1,credit_card,MA,7.46,5.0,21.0,1,659.0,655.0,0,0.0
12937,15000.0,36 months,11.49,494.57,B,1 year,OWN,70000.0,Not Verified,1,debt_consolidation,CA,27.04,19.0,30.0,0,669.0,665.0,0,0.0
12938,12000.0,36 months,13.99,410.08,C,1 year,RENT,47000.0,Source Verified,1,credit_card,MT,31.79,13.0,28.0,1,754.0,750.0,0,0.0


In [12]:
data['addr_state'].unique()

array(['MA', 'MN', 'TX', 'MS', 'IN', 'OH', 'FL', 'CA', 'AL', 'NY', 'SC',
       'IL', 'TN', 'NH', 'WA', 'WV', 'VA', 'PA', 'DE', 'RI', 'MD', 'MI',
       'MO', 'NJ', 'CO', 'NE', 'NC', 'UT', 'AR', 'LA', 'GA', 'WI', 'AZ',
       'MT', 'NV', 'KY', 'KS', 'ND', 'NM', 'OR', 'DC', 'HI', 'CT', 'OK',
       'AK', 'SD', 'ME', 'WY', 'VT', 'ID'], dtype=object)

In [14]:
def onehot_encode(data,column):
    data=data.copy()
    dummies=pd.get_dummies(data[column],prefix=column,dtype=int)
    data=pd.concat([data,dummies],axis=1)
    data=data.drop(column,axis=1)
    return data



In [16]:
data =onehot_encode(data,'addr_state')
data

,loan_amnt,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,...,addr_state_SD,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY
0,7200.0,36 months,17.27,257.67,D,9 years,OWN,20000.0,Verified,0,...,0,0,0,0,0,0,0,0,0,0
1,16000.0,36 months,5.32,481.84,A,8 years,MORTGAGE,96000.0,Not Verified,1,...,0,0,0,0,0,0,0,0,0,0
2,8000.0,36 months,7.91,250.36,A,5 years,OWN,66500.0,Verified,1,...,0,0,1,0,0,0,0,0,0,0
3,10000.0,36 months,8.49,315.63,B,4 years,OWN,35000.0,Not Verified,1,...,0,0,0,0,0,0,0,0,0,0
4,24000.0,60 months,13.99,558.32,C,10+ years,MORTGAGE,115000.0,Verified,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12935,6000.0,36 months,9.49,192.17,B,10+ years,MORTGAGE,50000.0,Source Verified,1,...,0,0,0,0,1,0,0,0,0,0
12936,12000.0,60 months,14.49,282.28,C,10+ years,OWN,122000.0,Not Verified,1,...,0,0,0,0,0,0,0,0,0,0
12937,15000.0,36 months,11.49,494.57,B,1 year,OWN,70000.0,Not Verified,1,...,0,0,0,0,0,0,0,0,0,0
12938,12000.0,36 months,13.99,410.08,C,1 year,RENT,47000.0,Source Verified,1,...,0,0,0,0,0,0,0,0,0,0


In [18]:
data['purpose'].unique()

array(['debt_consolidation', 'home_improvement', 'credit_card',
       'major_purchase', 'medical', 'other', 'car', 'small_business',
       'house', 'moving', 'vacation', 'renewable_energy', 'wedding',
       'educational'], dtype=object)

In [20]:
data=onehot_encode(data,'purpose')

In [22]:
data['verification_status']=data['verification_status'].replace({'Not Verified':0, 'Source Verified':1, 'Verified':2})

C:\Users\shrey\AppData\Local\Temp\ipykernel_7256\2709066839.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['verification_status']=data['verification_status'].replace({'Not Verified':0, 'Source Verified':1, 'Verified':2})


In [24]:
data['home_ownership']=data['home_ownership'].replace({'MORTGAGE':0, 'RENT':1, 'OWN':2, 'ANY':3, 'NONE':4, 'OTHER':4})

C:\Users\shrey\AppData\Local\Temp\ipykernel_7256\1393034601.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['home_ownership']=data['home_ownership'].replace({'MORTGAGE':0, 'RENT':1, 'OWN':2, 'ANY':3, 'NONE':4, 'OTHER':4})


In [26]:
data['emp_length']=data['emp_length'].replace({'10+ years':10, '3 years':3, '6 years':6, '1 year':1, '7 years':7, '8 years':8,
       '5 years':5, '2 years':2, '9 years':9, '< 1 year':0, '4 years':4})

C:\Users\shrey\AppData\Local\Temp\ipykernel_7256\3214867366.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['emp_length']=data['emp_length'].replace({'10+ years':10, '3 years':3, '6 years':6, '1 year':1, '7 years':7, '8 years':8,


In [28]:
data['term']=data['term'].replace({' 36 months':0, ' 60 months':1})

C:\Users\shrey\AppData\Local\Temp\ipykernel_7256\1918118054.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['term']=data['term'].replace({' 36 months':0, ' 60 months':1})


In [30]:
for column in data.columns:
    if data[column].dtype=='bool':
        data[column]=data[column].astype(int)

In [32]:
[column for column in data.columns if data[column].dtype=='object']

['grade']

In [34]:
data=onehot_encode(data,'grade')
data

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,dti,...,purpose_small_business,purpose_vacation,purpose_wedding,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G
0,7200.0,0,17.27,257.67,9,2,20000.0,2,0,13.21,...,0,0,0,0,0,0,1,0,0,0
1,16000.0,0,5.32,481.84,8,0,96000.0,0,1,14.45,...,0,0,0,1,0,0,0,0,0,0
2,8000.0,0,7.91,250.36,5,2,66500.0,2,1,29.22,...,0,0,0,1,0,0,0,0,0,0
3,10000.0,0,8.49,315.63,4,2,35000.0,0,1,33.54,...,0,0,0,0,1,0,0,0,0,0
4,24000.0,1,13.99,558.32,10,0,115000.0,2,1,17.71,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12935,6000.0,0,9.49,192.17,10,0,50000.0,1,1,12.48,...,0,0,0,0,1,0,0,0,0,0
12936,12000.0,1,14.49,282.28,10,2,122000.0,0,1,7.46,...,0,0,0,0,0,1,0,0,0,0
12937,15000.0,0,11.49,494.57,1,2,70000.0,0,1,27.04,...,0,0,0,0,1,0,0,0,0,0
12938,12000.0,0,13.99,410.08,1,1,47000.0,1,1,31.79,...,0,0,0,0,0,1,0,0,0,0


In [36]:
from sklearn.model_selection import train_test_split
y=data['loan_status']
x=data.drop('loan_status',axis=1)
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.8,random_state=123,shuffle=True)


In [38]:
x_train.shape

(10352, 87)

In [40]:
x_test.shape

(2588, 87)

In [42]:
scaler=StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)


In [108]:
svm_rbf=SVC(kernel='rbf',probability=True,random_state=42,C=1,class_weight='balanced')
svm_rbf.fit(x_train,y_train)

SVC(C=1, class_weight='balanced', probability=True, random_state=42)

In [110]:
y_pred = svm_rbf.predict(x_test)
y_pred

array([1, 0, 1, ..., 1, 1, 1], dtype=int64)

In [112]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 87.91%
Classification Report:
               precision    recall  f1-score   support

           0       0.68      0.84      0.75       557
           1       0.95      0.89      0.92      2031

    accuracy                           0.88      2588
   macro avg       0.81      0.87      0.84      2588
weighted avg       0.89      0.88      0.88      2588



In [114]:
scores = cross_val_score(svm_rbf, x, y, cv=5)
scores

array([0.5927357 , 0.58578053, 0.58616692, 0.5950541 , 0.59080371])

In [119]:
np.std(scores)

0.0036362566788311016